In [1]:
%run "../Libraries/ADP_Unzip"

In [2]:
process = "FMT_ING"
logger_name = "FMT_File_MoveToLanding_Unzip"
level_action = "DEFAULT"
log_level = "DEFAULT"

In [3]:
#################################################################################
""" Unzip Farmatic files from Ingestion (/mnt/blob/ingestion/pharmatic) to Landing (/mnt/blob/landing/pharmatic)

"""
 #Who                 When           What
 #Victor Salesa       05/11/2018     Initial Version
 #Ana Perez           07/02/2019     Replace dbutils.fs.ls function to blob_ls function and adapt code from new type generated
 #Victor Salesa Sanz  12/02/2019     Change code to include archiving when unzipping 
 #Victor Salesa Sanz  20/02/2019     Change code to include zip password as a parameter
 #Victor Salesa Sanz  25/02/2019     Correct /dbfs instead of /dbfs/ and files_unzipped_new to files_unzipped
 #Victor Salesa       26/02/2019     Added dbutils.notebook.exit 
 #Victor Salesa       04/03/2019     Added internal_call parameter to avoid dbutils.notebook.exit from inside
 #Ana Perez           27/03/2019     Included log managment and exception managment
 #Victor Salesa       04/03/2019     Splitted code into UnzipPharmaticFilesFromIngestion,MoveUncompressedFilesToArchive,StoreUnzipResultsToCSV 
 #                                   for better code understanding and isolated Exception and Error control of the different blocks.
 #Ana Perez           06/05/2019     Changed ADP_Farmatic_Ingestion_Process library by ADP_Unzip library
 #Victor Salesa       07/05/2019     Changed Delete files function to DeleteFilesFromLandingExcludingPattern
################################################################################

try:
  
  #Create parameter to launch task from notebook launcher
    try:
      level_action = dbutils.widgets.get("level_action") 
      log_level = dbutils.widgets.get("log_level")  
      ADP_log_info(process, logger_name, level_action, log_level, "BEGIN", sys._getframe().f_code.co_name)   
    except Exception as err:
      ADP_log_info(process, logger_name, "DEFAULT", "DEFAULT", "Internal Call parameter was not Created Please Re-run", sys._getframe().f_code.co_name)   

      dbutils.widgets.text("level_action", "DEFAULT","Level Action")
      dbutils.widgets.text("log_level","DEFAULT","Log Level")

      level_action = dbutils.widgets.get("level_action") 
      log_level = dbutils.widgets.get("log_level")

    #end exception 

    #Define folders
    ingestion_folder     = __PHARMATIC_INGESTION_BASE_PATH__
    archiving_folder     = __PHARMATIC_ARCHIVE_BASE_PATH__
    uncompress_folder    = "/dbfs"+__PHARMATIC_LANDING_BASE_PATH__
    
    ADP_log_debug(process, logger_name, level_action, log_level, "Before read blob_ls zip_list" , sys._getframe().f_code.co_name)
    
    #List files in ingestion folder
    file_list = blob_ls(ingestion_folder)
    
    if len(file_list)!=0:

      zip_list_df = UnzipPharmaticFilesFromIngestion(file_list,uncompress_folder,__FARMATIC_ZIP_PASSWD__)
      
      zip_files_count = callSafe(zip_list_df,"count",0)
      zip_files_uncompressed_count = callSafe(
                                          callSafe(zip_list_df,"filter",0,(col("uncompress_result") == 1))
                                          ,"count",0)
      if zip_files_count!=0:
        zip_files_moved_df = MoveUncompressedFilesToArchive(zip_list_df,archiving_folder)

        zip_files_moved_count = callSafe(zip_files_moved_df,"count",0)        
        
        if zip_files_moved_count != 0:
           StoreUnzipResultsToCSV(zip_files_moved_df,__INGESTION_BASE_PATH__)
        
        if zip_files_uncompressed_count!=0:
           DeleteFilesFromLandingExcludingPattern()

        ADP_log_info(process, logger_name, level_action, log_level, "END", sys._getframe().f_code.co_name) 
      else:
        ADP_log_warning(process, logger_name, level_action, log_level, "END No zip files found", sys._getframe().f_code.co_name)
    else:
      ADP_log_warning(process, logger_name, level_action, log_level, "END No files found", sys._getframe().f_code.co_name)
    #end len(file_list)!=0:   
except Exception as err:
  ADP_log_exception(process, logger_name, level_action, log_level,  "", sys._getframe().f_code.co_name,  sys.exc_info())
  raise Exception(err)